# Copyright
<pre>
You may use and modify this code for research and development purpuses.
Using this code for educational purposes (self-paced or instructor led) without the permission of the author is prohibited.
Copyright (c) 2023 Bálint Gyires-Tóth - All Rights Reserved

</pre>

In this notebook, we will classify human activities based on smartphone accelerometer and gyroscope data. The corresponding Human Activity Recognition Using Smartphones Data Set is delivered by UCI Machine Learning Repository, for more information [please visit the site](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones).

## 1. Data acquisition
Lets download and unpack the data.

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip

--2023-04-12 20:48:54--  https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60999314 (58M) [application/x-httpd-php]
Saving to: 'UCI HAR Dataset.zip'

     0K .......... .......... .......... .......... ..........  0% 97.0K 10m14s
    50K .......... .......... .......... .......... ..........  0%  302K 6m45s
   100K .......... .......... .......... .......... ..........  0% 8.35M 4m32s
   150K .......... .......... .......... .......... ..........  0% 10.0M 3m25s
   200K .......... .......... .......... .......... ..........  0%  152K 4m2s
   250K .......... .......... .......... .......... ..........  0% 9.86M 3m23s
   300K .......... .......... .......... .......... ..........  0% 10.6M 2m54s
   350K .......... .......... .......... ...

In [2]:
!unzip "UCI HAR Dataset.zip" > null

'unzip' is not recognized as an internal or external command,
operable program or batch file.


## 2. Imports
In this notebook we will need the following packages.

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
import seaborn as sns

## 3. Loading the data
The following functions will load the training and test data from the unpacked dataset.

In [4]:
# this function loads a single file (input or output)
def load_file(filepath):
  df = pd.read_csv(filepath, header=None, delim_whitespace=True)
  return df.values

# this function loads a set of files into a Numpy array with a shape of [samples, timesteps, features]
def load_group(filenames, prefix=''):
  loaded = list()
  for name in filenames:
    data = load_file(prefix + name)
    loaded.append(data)
    # stack group so that features are the 3rd dimension
  loaded = np.dstack(loaded)
  return loaded

 # load inputs and outputs from a directory
def load_XY(group, path):
  filepath = path + group + '/Inertial Signals/'
  filenames = list()
  filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt'] + \
               ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt'] + \
               ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
  # inputs
  X = load_group(filenames, filepath)
  # output (class)
  Y = load_file(path + group + '/y_'+group+'.txt')
  return X, Y

# load the dataset, returns train and test X and y elements
def load_dataset(path='UCI HAR Dataset/'):
  # load all train
  X_train, Y_train = load_XY('train', path)
  X_test,  Y_test  = load_XY('test', path)
  Y_train = to_categorical(Y_train - 1 ) # we need -1 to scale the categories between 0..5 (from 1..6)
  Y_test  = to_categorical(Y_test - 1 ) # we need -1 to scale the categories between 0..5 (from 1..6)
  return X_train, Y_train, X_test, Y_test

In [5]:
X_train, Y_train, X_test, Y_test = load_dataset()

FileNotFoundError: [Errno 2] No such file or directory: 'UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt'

In [ ]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
Y_train[0],

## 3.1. Data exploration: target variable - exercise 1
Let's check how many data instances there are for each class:

In [ ]:
print("No. of instance belong to class 0:",len(<TODO>))
print("No. of instance belong to class 1:",len(<TODO>))
print("No. of instance belong to class 2:",len(<TODO>))
print("No. of instance belong to class 3:",len(<TODO>))
print("No. of instance belong to class 4:",len(<TODO>))
print("No. of instance belong to class 5:",len(<TODO>))

You can doublecheck your calculations by plotting the histogram of the target variables: 

In [ ]:
sns.histplot(np.argmax(Y_train,axis=1))

## 3.2. Data exploration: input variables - exercise 2
And now let's select one instance from each class and plot the corresponding timeseries:

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12, 6))

for i in range(2): # row
  for k in range(3): # column
    for l in range(X_train.shape[2]): # dimensons to be displayed
      class_no=i*2+k
      axs[i,k].plot(<TODO: select the input sequence of one instance from class #class_no>)
      axs[i,k].title.set_text("Class "+str(class_no))
plt.legend()

## 4. Model definition and training - exercise 3

Let's define and train a 1D convolutional neural network, with the multidimensional timeseries as inputs, and the class in one-hot encoding as output. 

This time your task is to correct the code below and achieve higher than 90% validation accuracy and test accuracy. The test accuracy is calculated in section 5. 
Currently, **there are many errors (both syntactical and theroetical) in the model definition, compile and fit parts**, that should be fixed to achieve the goal. Don't change the number of layers and the layer types, and validation_split, just the other hyperparameters. There are several good solutions!

Hints: 
* use narrow and deep Conv1D kernels
* use standard activation functions for the inner layers (e.g. relu, sigmoid, tanh) with the corresponding initialization method
* use the appropriate activation function the output and the corresponding loss function (which is aligned with the classification task)
* use regularization, but not too agressive
* use standard optimizers
* train in mini-batches

In [ ]:
nb_filter       = 3   # number of filter/kernels in the convolutional layers
filter_length   = 48  # length of the filter/kernel in the convolutional layers
window_size     = X_train.shape[1] # the window size defined by the dataset
nb_features     = X_train.shape[2] # the number of features of the input data
nb_outputs      = Y_train.shape[1] # the number of outputs (defined by the target data)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, activation='selu', kernel_initializer='he_normal', input_shape=(window_size, nb_features)))
model.add(MaxPooling1D())
model.add(Dropout(0.99))
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, activation='selu', kernel_initializer='he_normal'))
model.add(MaxPooling1D())
model.add(Dropout(0.99))
model.add(Flatten())
model.add(Dense(128, activation='selu'))
model.add(Dropout(0.99))
model.add(Dense(nb_outputs, activation='linear'))

Compiling the model with the appropriate loss function for multiclass classification.

In [ ]:
model.compile(loss='mse', optimizer='david', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
es = EarlyStopping(patience=-1, restore_best_weights=True, monitor="validation_loss")

And now we run the training:

In [ ]:
model.fit(X_train, Y_train, 
          epochs=200, batch_size=1, 
          validation_split=0.2, # don't change the validation split
          callbacks=[es],
          verbose=2)

## 5. Evaluation
Please run the cells below to check the test loss and test accuracy (remember, the accuracy must be over 90%) and inspect the other evaluation methods as well. 

In [ ]:
eval = model.evaluate(X_test,Y_test)
print("Test loss:",eval[0])
print("Test accuracy:",eval[1])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
preds = model.predict(X_test)

In [ ]:
print(classification_report(np.argmax(Y_test,1),np.argmax(preds,1)))

In [ ]:
conf=confusion_matrix(np.argmax(Y_test,1),np.argmax(preds,1))
sns.heatmap(conf, annot=True, fmt='d', vmax=100)